<a href="https://colab.research.google.com/github/andrepporto/IFRO-DataScience/blob/main/catsanddogsVGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import os
import keras
import tensorflow as tf
from keras import layers
from tensorflow import data as tf_data
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model


In [3]:
!mkdir dataset
os.chdir("dataset")

In [4]:
!pwd

/content/dataset


In [5]:
!curl -L -o dataset.zip https://www.kaggle.com/api/v1/datasets/download/aleemaparakatta/cats-and-dogs-mini-dataset
!unzip -q dataset.zip
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 21.8M  100 21.8M    0     0  19.9M      0  0:00:01  0:00:01 --:--:-- 39.6M
cats_set  dataset.zip  dogs_set


In [6]:
os.chdir("..")

In [7]:
num_skipped = 0
for folder_name in ("cats_set", "dogs_set"):
  folder_path = os.path.join("dataset", folder_name)
  for fname in os.listdir(folder_path):
    fpath = os.path.join(folder_path, fname)
    try:
      fobj = open(fpath, "rb")
      is_jfif = b"JFIF" in fobj.peek(10)
    finally:
      fobj.close()

    if not is_jfif:
      num_skipped += 1
      os.remove(fpath)

print(f"Deleted {num_skipped} images.")

Deleted 0 images.


In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image_dataset_from_directory

BATCH_SIZE = 32
IMG_SIZE = (224, 224)

train_dataset = image_dataset_from_directory(
    "dataset",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

val_dataset = image_dataset_from_directory(
    "dataset",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE
)

Found 1000 files belonging to 2 classes.
Using 800 files for training.
Found 1000 files belonging to 2 classes.
Using 200 files for validation.


In [9]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras import layers, models

base_model = MobileNetV2(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Congela os pesos do modelo pré-treinado

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation="relu"),
    layers.Dropout(0.5),
    layers.Dense(1, activation="sigmoid")  # Saída binária (0 = gato, 1 = cachorro)
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

# Treinamento
model.fit(train_dataset, validation_data=val_dataset, epochs=5)

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 50s 2s/step - accuracy: 0.5543 - loss: 0.8791 - val_accuracy: 0.6950 - val_loss: 0.6117
Epoch 2/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.6414 - loss: 0.6274 - val_accuracy: 0.5350 - val_loss: 0.6645
Epoch 3/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 83s 2s/step - accuracy: 0.6432 - loss: 0.6251 - val_accuracy: 0.7250 - val_loss: 0.5973
Epoch 4/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 80s 2s/step - accuracy: 0.7504 - loss: 0.5375 - val_accuracy: 0.5550 - val_loss: 0.7079
Epoch 5/5
25/25 ━━━━━━━━━━━━━━━━━━━━ 88s 2s/step - accuracy: 0.7095 - loss: 0.5422 - val_accuracy: 0.7350 - val_loss: 0.5667


In [10]:
model.save("modelo_cachorro_gato.h5")

In [11]:
!pip install streamlit tensorflow pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 94.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 6.5 MB/s eta 0:00:00


In [12]:
import streamlit as st
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.vgg16 import preprocess_input
import numpy as np
from PIL import Image

# Carregar modelo treinado
model = load_model("modelo_cachorro_gato.h5")

# Interface do Usuário
st.title("Classificador de Gatos e Cachorros 🐶🐱")
uploaded_file = st.file_uploader("Envie uma imagem...", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption="Imagem carregada", use_column_width=True)

    # Preprocessamento
    img = image.resize((224, 224))
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    # Predição
    prediction = model.predict(img_array)
    label = "Cachorro 🐶" if prediction[0][0] >= 0.5 else "Gato 🐱"

    st.write(f"### Resultado: {label}")

2025-02-13 23:58:59.183 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 23:58:59.357 
  command:

    streamlit run /usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-02-13 23:58:59.359 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 23:58:59.363 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 23:58:59.365 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 23:58:59.368 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 23:58:59.371 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-02-13 23:58:59.374 Thread 'MainThread': mi

In [ ]:
keras.applications.VGG16(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax",
    name="vgg16",
)
